In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# import dataset

In [12]:
# kaggle online path
# base_path = "../input/competitive-data-science-predict-future-sales/"

# local path
base_path = os.path.join(os.getcwd(), "dataset")

In [16]:
item_categories_path = os.path.join(base_path, "item_categories.csv")
items_path = os.path.join(base_path, "items.csv")
sales_train_path = os.path.join(base_path, "sales_train.csv")
shops_path = os.path.join(base_path, "shops.csv")
test_path = os.path.join(base_path, "test.csv")
sample_submission_path = os.path.join(base_path, "sample_submission.csv")

In [18]:
item_categories_df = pd.read_csv(item_categories_path)
shops_df = pd.read_csv(shops_path)
items_df = pd.read_csv(items_path)

sales_df = pd.read_csv(sales_train_path)

test_df = pd.read_csv(test_path)

sample_submission_df = pd.read_csv(sample_submission_path)

In [19]:
items_df = pd.read_csv(items_path)
item_categories_df

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
...,...,...
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82


1. 가게별로 아이템 가격이 다름
2. 같은 가게, 같은 아이템이라도 월별로 가격이 달라지는 경우도 있음

# Preprocessing

## 1. separate city name

In [20]:
shops_df['city'] = shops_df['shop_name'].str.split(" ").str[0]
shops_df['shop_detail'] = shops_df['shop_name'].str.split(" ").str[-1].str.replace('"', '')
shops_df.head()

,shop_name,shop_id,city,shop_detail
0,"!Якутск Орджоникидзе, 56 фран",0,!Якутск,фран
1,"!Якутск ТЦ ""Центральный"" фран",1,!Якутск,фран
2,"Адыгея ТЦ ""Мега""",2,Адыгея,Мега
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,Октябрь-Киномир
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,Молл


## 2. separate item category groups

In [21]:
item_categories_df['item_category_group'] = item_categories_df['item_category_name'].str.split(" - ").str[0].str.split('(').str[0]
item_categories_df['item_category_sub'] = item_categories_df['item_category_name'].str.split(" - ").str[-1].str.split('(').str[-1].str.split(')').str[0]
item_categories_df.head()

,item_category_name,item_category_id,item_category_group,item_category_sub
0,PC - Гарнитуры/Наушники,0,PC,Гарнитуры/Наушники
1,Аксессуары - PS2,1,Аксессуары,PS2
2,Аксессуары - PS3,2,Аксессуары,PS3
3,Аксессуары - PS4,3,Аксессуары,PS4
4,Аксессуары - PSP,4,Аксессуары,PSP


## 3. clean item name

In [22]:
items_df['item_name_clean'] = items_df['item_name'].str.split('[').str[0].str.split('(').str[0].str.split('-').str[0].str.replace('*','').str.replace('/','')
items_df.drop(['item_name'], axis=1, inplace=True)
items_df.head()

,item_id,item_category_id,item_name_clean
0,0,40,! ВО ВЛАСТИ НАВАЖДЕНИЯ
1,1,76,!ABBYY FineReader 12 Professional Edition Full
2,2,40,В ЛУЧАХ СЛАВЫ
3,3,40,ГОЛУБАЯ ВОЛНА
4,4,40,КОРОБКА


## 4. get montly data from sales_df

In [23]:
sales_df['month'] = sales_df['date'].str.split(".").str[1].apply(pd.to_numeric)
sales_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month
0,02.01.2013,0,59,22154,999.00,1.0,1
1,03.01.2013,0,25,2552,899.00,1.0,1
2,05.01.2013,0,25,2552,899.00,-1.0,1
3,06.01.2013,0,25,2554,1709.05,1.0,1
4,15.01.2013,0,25,2555,1099.00,1.0,1


In [24]:
sales_df.groupby([sales_df['item_id'],sales_df['shop_id'], sales_df['date_block_num']]).sum().head(20)

date_block_num  shop_id  item_id  item_price  \
item_id shop_id date_block_num                                                 
0       54      20                          20       54        0        58.0   
1       55      15                          30      110        2      8980.0   
                18                          18       55        1      4490.0   
                19                          19       55        1      4490.0   
                20                          20       55        1      4490.0   
                21                          21       55        1      4490.0   
2       54      19                          19       54        2        58.0   
                22                          22       54        2        58.0   
3       54      18                          18       54        3       100.0   
                19                          19       54        3        58.0   
4       54      20                          20       54        4        58.0   
5       54      23                          23       54        5        28.0   
6       54      18                          18       54        6       100.0   
7       54      23                          23       54        7        28.0   
8       54      19                          19       54        8        58.0   
                20                          20       54        8        58.0   
9       54      19                          19       54        9        58.0   
10      54      24                          24       54       10        28.0   
11      54      22                          22       54       11        58.0   
12      54      1                            1       54       12       189.0   

                                item_cnt_day  month  
item_id shop_id date_block_num                       
0       54      20                       1.0      9  
1       55      15                       2.0      8  
                18                       1.0      7  
                19                       1.0      8  
                20                       1.0      9  
                21                       1.0     10  
2       54      19                       1.0      8  
                22                       1.0     11  
3       54      18                       1.0      7  
                19                       1.0      8  
4       54      20                       1.0      9  
5       54      23                       1.0     12  
6       54      18                       1.0      7  
7       54      23                       1.0     12  
8       54      19                       1.0      8  
                20                       1.0      9  
9       54      19                       1.0      8  
10      54      24                       1.0      1  
11      54      22                       1.0     11  
12      54      1                        1.0      2

In [38]:
sales_df_block_num = sales_df.groupby([sales_df['item_id'],sales_df['shop_id'], sales_df['date_block_num']]).sum()
sales_df_block_num.drop(['item_price', 'month'], axis=1, inplace=True)
sales_df_block_num = sales_df_block_num.reset_index()
sales_df_block_num.rename(columns={"item_cnt_day": "item_count_month"}, inplace=True)
sales_df_block_num
print(sales_df_block_num.shape)
sales_df_block_num.head()

(1609124, 4)


,item_id,shop_id,date_block_num,item_count_month
0,0,54,20,1.0
1,1,55,15,2.0
2,1,55,18,1.0
3,1,55,19,1.0
4,1,55,20,1.0


In [43]:
sales_df_month = sales_df.groupby([sales_df['shop_id'], sales_df['item_id'], sales_df['month']]).sum().add_suffix('_Sum').reset_index()
sales_df_month.drop(['date_block_num_Sum', 'item_price_Sum'], axis=1, inplace=True)
sales_df_month = sales_df_month.groupby([sales_df_month['shop_id'], sales_df_month['item_id']]).mean().reset_index()
sales_df_month.drop(['month'], axis=1, inplace=True)
sales_df_month.rename(columns={"item_cnt_day_Sum": "item_count_month"}, inplace=True)
print(sales_df_month.shape)
sales_df_month.head()

(424124, 3)


,shop_id,item_id,item_count_month
0,0,30,31.0
1,0,31,11.0
2,0,32,8.0
3,0,33,3.0
4,0,35,7.5


## finite dataset

In [28]:
sales_df_month = sales_df_month.merge(shops_df, how='left').drop(['shop_id', 'shop_name'], axis=1)
sales_df_month = sales_df_month.merge(items_df, how='left').drop(['item_id'], axis=1)
sales_df_month = sales_df_month.merge(item_categories_df, how='left').drop(['item_category_id', 'item_category_name'], axis=1)
sales_df_month

,item_count_month,city,shop_detail,item_name_clean,item_category_group,item_category_sub
0,31.000000,!Якутск,фран,007: КООРДИНАТЫ «СКАЙФОЛЛ»,Кино,DVD
1,31.000000,!Якутск,фран,007: КООРДИНАТЫ «СКАЙФОЛЛ»,Кино,DVD
2,11.000000,!Якутск,фран,007: КООРДИНАТЫ «СКАЙФОЛЛ»,Кино,Blu-Ray
3,11.000000,!Якутск,фран,007: КООРДИНАТЫ «СКАЙФОЛЛ»,Кино,Blu-Ray
4,8.000000,!Якутск,фран,1+1,Кино,DVD
...,...,...,...,...,...,...
453656,1.000000,Ярославль,Альтаир,ЯВЛЕНИЕ 2012,Кино,Blu-Ray
453657,1.000000,Ярославль,Альтаир,"ЯДЫ, ИЛИ ВСЕМИРНАЯ ИСТОРИЯ ОТРАВЛЕНИЙ",Кино,Blu-Ray
453658,3.200000,Ярославль,Альтаир,ЯРОСТЬ,Кино,DVD
453659,1.500000,Ярославль,Альтаир,ЯРОСТЬ,Кино,Blu-Ray


### get dummies

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
sales_df_block_num.head()

,item_id,shop_id,date_block_num,item_count_month
0,0,54,20,1.0
1,1,55,15,2.0
2,1,55,18,1.0
3,1,55,19,1.0
4,1,55,20,1.0


In [44]:
train_df = sales_df_month

y = train_df['item_count_month']
X = train_df.drop(['item_count_month'], axis=1)

# X = pd.get_dummies(X, columns = ['item_id', 'shop_id'], drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)


(339299, 2) (84825, 2)
(339299,) (84825,)


# Create models

In [40]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

In [49]:
xgb = XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=6)
xgb.fit(X_train, y_train, eval_metric="rmse")

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [50]:
xgb_pred = xgb.predict(X_test)

np.sqrt(mean_squared_error(y_test, xgb_pred))

5.775837852163363